In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torch.utils.data as data
from torch.utils.data import DataLoader as DataLoader
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.over_sampling import ADASYN

In [2]:
torch.set_default_tensor_type(torch.FloatTensor)

In [3]:
test_data = pd.read_csv('data/bd/test_data_eda_1.csv')
del test_data['Unnamed: 0']
del test_data['customer_id']
test_x = test_data.values

In [4]:
train_x = pd.read_csv('data/bd/x_new.csv')
#保存文件时忘记了index = False
del train_x['Unnamed: 0']
train_y = pd.read_csv('data/bd/y_new.csv')
del train_y['0']


In [5]:
X_new = train_x.values
Y_new = train_y.values

In [6]:
train_index = []
p = 0
total_zero_count = 0
total_one_count = 0
target_zero_count = len(Y_new)/5*2
target_one_count = len(Y_new)/5*2
while True:
    if p %100000 == 0:
        print("now dealing count :",p)
    if total_zero_count >= target_zero_count and total_one_count >= target_one_count:
        break
    if Y_new[p] == 0:
        if total_zero_count < target_zero_count:
            train_index.append(p)
            total_zero_count += 1
            p += 1
            continue
        else:
            p += 1
            continue
    else:
        if total_one_count < target_one_count:
            train_index.append(p)
            total_one_count += 1
            p += 1
            continue
        else:
            p += 1
            continue
    

now dealing count : 0
now dealing count : 100000
now dealing count : 200000
now dealing count : 300000
now dealing count : 400000
now dealing count : 500000


In [7]:
total_index = [x for x in range(len(Y_new))]
for i in train_index:
    total_index.remove(i)

In [8]:
valid_index = total_index

In [9]:
train_x =  X_new[train_index]
train_y =  Y_new[train_index]
valid_x =  X_new[valid_index]
valid_y =  Y_new[valid_index]

In [10]:
class bd_dataset(data.Dataset):     
    def __init__(self, mode,data_x,data_y = None):          
        self.mode = mode
        self.data_x = data_x
        self.data_y = data_y
        self.data_size =len(data_y)                 

    def __getitem__(self, item):            
        if self.mode == 'train' or self.mode == 'valid':                                        
            item_x = self.data_x[item]  
            item_y = self.data_y[item]                             
            return Variable(torch.tensor(item_x,dtype = torch.float32)), torch.tensor(item_y,dtype = torch.float32)    
        elif self.mode == 'test':                                      
            item_x = self.data_x[item]
            return Variable(torch.tensor(item_x,dtype = torch.float32))                                      
        else:
            print('Mode Worry')

    def __len__(self):
        return self.data_size               # 返回数据集大小


In [11]:
class fcnn(nn.Module):
    def __init__(self):
        super(fcnn,self).__init__()
        self.bn = nn.BatchNorm1d(1, affine=False)
        self.fc1 = nn.Linear(65,128)
        self.fc2 = nn.Linear(128,60)
        self.fc3 = nn.Linear(60,20)
        self.fc4 = nn.Linear(20,1)
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.sigmoid(self.fc4(x))
        return x


In [12]:
#上采样之后，类别不平衡消失，使用ACC作为指标
def test(model,batch_size,criterion):
    datafile = bd_dataset('valid', valid_x, valid_y)                                                      
    dataloader = DataLoader(datafile, batch_size=32, shuffle=True) 
    all_loss = []
    for i, j in dataloader:
        train_x_temp = i.to(device)
        labels = j.to(device)
        outputs = model(train_x_temp)
        loss = criterion(outputs, labels)
        loss_cpu = np.mean(loss.to('cpu').detach().numpy())
        all_loss.append(loss_cpu)
    return np.mean(all_loss)

torch.save(fc_net,'data/bd/model_epoch_0.pth')

In [31]:
fc_net = torch.load('data/bd/model_epoch_0.pth')

In [33]:
#criterion = nn.CrossEntropyLoss()

#criterion = nn.CrossEntropyLoss(weight=torch.tensor([1,1],dtype = torch.float32).cuda())
criterion = nn.MSELoss(reduce=False, size_average=False)
optimizer = optim.Adam(fc_net.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)
#optimizer = torch.optim.SGD(fc_net.parameters(),lr=0.02)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [34]:
def train(train_data, train_data_y, model, criterion, optimizer,epochs, batch_size):
    valid_loss = []
    datafile = bd_dataset('train', train_x, train_y)                                                      
    dataloader = DataLoader(datafile, batch_size=32, shuffle=True) 
    all_epoch_loss = []
    
    for epoch in range(epochs): 
        epoch_valid_loss = test(fc_net,32,criterion)
        valid_loss.append(epoch_valid_loss)
        temp_epoch_loss = 0
        print('-------------------------------------------------')
        print('epoch: ',epoch)
        iter_count = 0
        for i, j in dataloader:
            iter_count += 1
            
            
            train_x_temp = i.to(device)
            labels = j.to(device)
            outputs = model(train_x_temp)
            
            optimizer.zero_grad()        
            loss = criterion(outputs, labels)
            loss.backward(loss.clone().detach())
            loss_cpu = loss.to('cpu').detach().numpy()
            temp_epoch_loss += np.sum(loss_cpu)
            optimizer.step()
            
        print('loss: ',temp_epoch_loss/len(train_data))   
        
        all_epoch_loss.append(temp_epoch_loss)
        
    return valid_loss
        
        

In [35]:
a = train(X_new, Y_new, fc_net, criterion, optimizer,5, 32)

-------------------------------------------------
epoch:  0
loss:  0.18161994692502767
-------------------------------------------------
epoch:  1
loss:  0.18116499387238
-------------------------------------------------
epoch:  2
loss:  0.18108667697921796
-------------------------------------------------
epoch:  3
loss:  0.18093498729537053
-------------------------------------------------
epoch:  4
loss:  0.18086679071696685


In [39]:
test_x = Variable(torch.tensor(test_x,dtype = torch.float32)).cuda()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


In [40]:
result = fc_net(test_x)

In [41]:
result = result.to('cpu').detach().numpy()

In [45]:
def turn_to_result(result):
    for i in range(len(result)):
        if result[i] > 0.5:
            result[i] = 1
        else:
            result[i] = 0

In [46]:
turn_to_result(result)

In [48]:
#最终结果：result

In [47]:
result

array([[0.],
       [1.],
       [0.],
       ...,
       [1.],
       [1.],
       [0.]], dtype=float32)

In [ ]:
#下面是训练时的代码
#当epoch = 5时，验证集loss最小

In [ ]:
a = train(train_x, train_y, fc_net, criterion, optimizer,10, 32)